In [263]:
import numpy as np


In [264]:
sigma = np.float64(5.67*10**(-8))
earth_radius = np.float64(6371000)
earth_mass = np.float64(5.976 * 10**24)
G = np.float64(6.67 * 10**(-11))
albedo_coef = 0.3
earth_radiation = 240

As = np.array([0.6] * 6) 
e = np.array([0.4] * 6) 
qs = np.float64(1367)
generationHeat = np.array([50] * 6)

width = 0.005
Lx = np.float64(1)
Ly = np.float64(1)
Lz = np.float64(1)
R_value = 2

c = 800
p = 2700

height_above_Earth = np.float64(500)*1000
labels = ["Зенит", "Надир", "Перпенд 1", "Перпенд 2", "Теневая", "Солнечная"]


In [265]:
radius = height_above_Earth + earth_radius # в метры
velocity = np.sqrt(G * earth_mass / radius) # скорость 
length = 2*np.pi*radius #длина орбиты
period = length/velocity #период полного оборота

angleSunSet = np.arccos(earth_radius/(earth_radius+height_above_Earth))
tDay = period * (np.pi/2+angleSunSet)/np.pi

angleSunSet, tDay, period


(0.38384819529001624, 3526.626721270226, 5668.153198351584)

In [266]:
areas = np.array([Ly*Lz, Ly*Lz,
                Lx*Lz, Lx*Lz,
                Lx*Ly, Lx*Ly])

areas


array([1., 1., 1., 1., 1., 1.])

In [267]:
mass = p*width*areas[:]
mass


array([13.5, 13.5, 13.5, 13.5, 13.5, 13.5])

In [268]:
Qav = np.zeros(6)
Qav += generationHeat
Qav

# От солнца
Qav[0]+=areas[0]*As[0]*qs/np.pi #зенит
Qav[1]+=areas[1]*As[1]*(1-np.cos(angleSunSet))*qs/np.pi #надир после 6 часов
Qav[5]+=areas[5]*As[5]*(1+np.sin(angleSunSet))*qs/np.pi #после до 6 часов

#От солнечного альбедо
Qav[1]+=As[1]*areas[1]*albedo_coef*qs/np.pi

#От земли
Qav[1]+=e[1]*areas[1]*earth_radiation*earth_radius**2/radius**2
for i in range(6):
    print(f"{labels[i]}:{str(Qav[i])}")


Зенит:311.0777686479451
Надир:229.85845294508442
Перпенд 1:50.0
Перпенд 2:50.0
Теневая:50.0
Солнечная:408.8491469715451


In [269]:
R = R_value * np.ones(shape=(6,6))
for i in range(6):
    R[i][i] = 10**46
for i in range(3):
    R[2*i+1][2*i] = 10**46
    R[2*i][2*i+1] = 10**46
R


array([[1.e+46, 1.e+46, 2.e+00, 2.e+00, 2.e+00, 2.e+00],
       [1.e+46, 1.e+46, 2.e+00, 2.e+00, 2.e+00, 2.e+00],
       [2.e+00, 2.e+00, 1.e+46, 1.e+46, 2.e+00, 2.e+00],
       [2.e+00, 2.e+00, 1.e+46, 1.e+46, 2.e+00, 2.e+00],
       [2.e+00, 2.e+00, 2.e+00, 2.e+00, 1.e+46, 1.e+46],
       [2.e+00, 2.e+00, 2.e+00, 2.e+00, 1.e+46, 1.e+46]])

In [270]:
Tav = 300*np.ones(6)

eps = 0.001
eps_curr = 100
iter = 0

alpha = 0.2

while eps < eps_curr:
    eps_curr = -1
    iter += 1
    for i in range(6):

        newT = (1.0-alpha)*Tav[i] + alpha*(Qav[i] + np.sum(Tav/R[i])) / (sigma*Tav[i]**3*areas[i]*e[i]+np.sum(1/R[i]))
        eps_curr = max(np.abs(newT - Tav[i]), eps_curr)
        Tav[i] = newT

print(f"Total: {iter} iteration")

for i in range(len(labels)):
    print(f"{labels[i]} средняя темпераутра: {Tav[i]:5.2f}K")


Total: 38 iteration
Зенит средняя темпераутра: 321.37K
Надир средняя темпераутра: 304.38K
Перпенд 1 средняя темпераутра: 271.23K
Перпенд 2 средняя темпераутра: 271.23K
Теневая средняя темпераутра: 262.89K
Солнечная средняя темпераутра: 341.77K


In [271]:
Qex = np.zeros(6)
Qex[0] += qs * As[0] * areas[0] * np.sqrt(2)/2
Qex[5] += qs * As[1] * areas[1] * np.sqrt(2.5)/2
Qex[1] += e[1]*areas[1]*earth_radiation*earth_radius**2/radius**2
Qex[1] += 2*albedo_coef*qs*As[1]*areas[1]/np.pi
Qex += generationHeat
Qex


array([629.96898193, 289.18325577,  50.        ,  50.        ,
        50.        , 698.42503422])

In [272]:
Qex - Qav


array([318.89121328,  59.32480282,   0.        ,   0.        ,
         0.        , 289.57588725])

In [273]:
tDayCalculation = tDay/2


In [274]:
m = np.zeros(R.shape)
for i in range(6):
    for j in range(6):
        if i == j:
            index = R[i] > 0
            m[i][i] = np.abs(np.sum(1/R[i][index])) + c*mass[i]/tDayCalculation
        else:
            if R[i][j] != 0:
                m[i][j] = -1/R[i][j]

d = Qex[:] - Qav[:] + Tav[:]*c*mass[:]/tDayCalculation


In [275]:
tDayCalculation


1763.313360635113

In [276]:
m


array([[ 8.12483308e+00, -1.00000000e-46, -5.00000000e-01,
        -5.00000000e-01, -5.00000000e-01, -5.00000000e-01],
       [-1.00000000e-46,  8.12483308e+00, -5.00000000e-01,
        -5.00000000e-01, -5.00000000e-01, -5.00000000e-01],
       [-5.00000000e-01, -5.00000000e-01,  8.12483308e+00,
        -1.00000000e-46, -5.00000000e-01, -5.00000000e-01],
       [-5.00000000e-01, -5.00000000e-01, -1.00000000e-46,
         8.12483308e+00, -5.00000000e-01, -5.00000000e-01],
       [-5.00000000e-01, -5.00000000e-01, -5.00000000e-01,
        -5.00000000e-01,  8.12483308e+00, -1.00000000e-46],
       [-5.00000000e-01, -5.00000000e-01, -5.00000000e-01,
        -5.00000000e-01, -1.00000000e-46,  8.12483308e+00]])

In [277]:
d


array([2287.21625763, 1923.59257189, 1661.25066377, 1661.25066377,
       1610.16652006, 2382.8346135 ])

In [278]:
from scipy import linalg
Tmax = linalg.solve(m, d)

Tmax


array([356.23036007, 311.47575623, 285.17778408, 285.17778408,
       274.36838894, 369.46795417])

In [294]:
Qex = generationHeat
Qex[1]+=e[1]*areas[1]*earth_radiation*earth_radius**2/radius**2

Qex


array([ 50, 296,  50,  50,  50,  50])

In [280]:
Qex - Qav


array([-261.07776865,  -97.85845295,    0.        ,    0.        ,
          0.        , -358.84914697])

In [281]:
tNight = (period - tDay)/2

m = np.zeros(R.shape)
for i in range(6):
    for j in range(6):
        if i == j:
            index = R[i] > 0
            m[i][i] = np.abs(np.sum(1/R[i][index]/2)) + c*mass[i]/tNight
        else:
            if R[i][j] != 0:
                m[i][j] = -1/R[i][j]/2

d = Qex[:] - Qav[:] + Tav[:]*c*mass[:]/tNight


In [282]:
from scipy import linalg
Tmin = linalg.solve(m, d)

Tmin


array([293.96305358, 293.22878397, 272.83542359, 272.83542359,
       264.72476991, 304.11570522])

In [283]:
labels_temp = ["avg", "min", "max"]
Temps = [Tav, Tmin, Tmax]
for i in range(len(labels)):
    print(f"{labels[i].capitalize()}")
    result = ""
    for j in range(len(Temps)):
        result +=f"{labels_temp[j]}:{Temps[j][i]:5.2f}K "
    print(result)


Зенит
avg:321.37K min:293.96K max:356.23K 
Надир
avg:304.38K min:293.23K max:311.48K 
Перпенд 1
avg:271.23K min:272.84K max:285.18K 
Перпенд 2
avg:271.23K min:272.84K max:285.18K 
Теневая
avg:262.89K min:264.72K max:274.37K 
Солнечная
avg:341.77K min:304.12K max:369.47K 


In [284]:
heat_capacity=mass*c


In [285]:
heat_capacity


array([10800., 10800., 10800., 10800., 10800., 10800.])

In [286]:
for i in range(6):
    for j in range(6):
        if i == j:
            index = R[i] > 0
            m[i][i] = np.abs(np.sum(1/R[i][index]/2)) + heat_capacity[i]/tNight
        else:
            if R[i][j] != 0:
                m[i][j] = -1/R[i][j]/2


In [290]:
%%timeit

for i in range(6):
    for j in range(6):
        if i == j:
            m[i][i] = np.abs(np.sum(1/R[i])) + c*mass[i]/tDayCalculation
        else:
            m[i][j] = -1/R[i][j]

d = Qex[:] - Qav[:] + Tav[:]*c*mass[:]/tDayCalculation


118 µs ± 356 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [288]:
m[np.diag_indices_from(m)]


array([8.12483308, 8.12483308, 8.12483308, 8.12483308, 8.12483308,
       8.12483308])

In [289]:
np.sum(1/R, axis=1) + c*mass[:]/tDayCalculation


array([8.12483308, 8.12483308, 8.12483308, 8.12483308, 8.12483308,
       8.12483308])

In [292]:
%%timeit

m = -1/R
m[np.diag_indices_from(m)] = np.sum(1/R, axis=1) + c*mass[:]/tDayCalculation


55.3 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
